# Iterative Solvers with Linear Operators

Here we demo how to solve data-sparse linear operators with an iterative solver.  Essentially we'd like to solve
$$Ax = b$$
for $x$, where $A$ may have some special structure

The two packages we'll use are
* [LinearOperators.jl](https://juliasmoothoptimizers.github.io/LinearOperators.jl/latest/index.html)
* [IterativeSolvers.jl](https://juliamath.github.io/IterativeSolvers.jl/latest/)

In the exercises, you can also try out
* [RandomizedLinAlg.jl](https://haampie.github.io/RandomizedLinAlg.jl/latest/)

Which allows you to use randomized algorithms on linear operators.  Note that this package is still not very well developed.

In [ ]:
using LinearOperators, IterativeSolvers

In [ ]:
n = 100
v = randn(n,1)
v = v/norm(v)
opV = LinearOperator(v)
A = v*v'
opA = opV * opV'
;

In [ ]:
x = randn(n)
@time b1 = A * x
@time b2 = opA*x
@show norm(b1-b2)
;

## Use with ARPACK wrapped functions

You can use a LinearOperator with Julia's interface to [ARPACK](http://www.caam.rice.edu/software/ARPACK/) via [eigs](https://docs.julialang.org/en/stable/stdlib/linalg/#Base.LinAlg.eigs-Tuple{Any}) and [svds](https://docs.julialang.org/en/stable/stdlib/linalg/#Base.LinAlg.svds)


In [ ]:
# note that for some operators, you may neet to set Hermitan flags
@show hermitian(opA)
opA.hermitian=true
opA.symmetric=true
@show hermitian(opA)
;

In [ ]:
@time (E, V) = eigs(opA; nev=2, which=:LM)
@time (E2, V2) = eigs(A; nev=2, which=:LM)
;

In [ ]:
E

## Use with IterativeSolvers package

In [ ]:
@show c = sqrt(2*log(n))
opA2 = (2*c)*opA + Diagonal(abs.(randn(n))) # spiked model
opA2.symmetric=true
opA2.hermitian=true
opA2

In [ ]:
x_true = randn(n)
b = opA2 * x_true
@time x_est_minres = minres(opA2, b)
@time x_est_cg = cg(opA2, b)
@show norm(b - opA2*x_est_minres)
@show norm(b - opA2*x_est_cg)
;

In [ ]:
A2 = full(opA2)
@time x_est = A2\b
@show norm(b - A2*x_est)
;

## Exercises/Extras

If you're interested, try out one or more of the following exercises:

1. Try out the randomized linear algebra package [RandomizedLinAlg.jl](https://haampie.github.io/RandomizedLinAlg.jl/latest/).  Try using the ```rnorms``` function to estimate the matrix norm.

2. Make a plot of how long it takes to solve $Ax = b$ for $A$ diagonal + rank-1, for various sizes of problems. Estimate how long it would take to solve the equivalent problem using the full matrix

3. You can also use sparse matrices as LinearOperators, and with iterative solvers.  Use [sprand](https://docs.julialang.org/en/stable/stdlib/arrays/#Base.SparseArrays.sprand) to generate sparse matrices of various sizes and try using [gmres](https://juliamath.github.io/IterativeSolvers.jl/latest/linear_systems/gmres.html) to solve some linear systems.

4. Check out the [tutorial](https://juliasmoothoptimizers.github.io/LinearOperators.jl/latest/tutorial.html#Using-functions-1) on how to use functions as linear operators